## CNN也可以做電影評論分析！
利用Convolution1D。
我們想要丟一段電影評論給神經網路，讓偉大的神經網路告訴我們是正評或負評(用0或1表示)。    
使用的資料庫是keras裡的IMDb --    
該資料庫的編法是將評論中最常用的字排序編號，因此你可以看到x_train.shape是一堆數字的array，並以0或1表示該評論為正評或負評。

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 讀入套件、準備資料    
我們這裡用的是Conv1D, GlobalMaxPooling1D訓練方式是adam。

In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

In [4]:
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


## 0. 修改資料格式
輸入資料需要做修改，輸出資料本來就已經是我們要的形式。
因為評論長度有長有短，但我們希望25000筆資料的長度都是一致的，因此我只取評論中的400字(maxlen)，太多就砍掉、太少就補0。

In [5]:
x_train.shape

(25000,)

In [6]:
x_train = sequence.pad_sequences(x_train, maxlen=400)
x_test = sequence.pad_sequences(x_test, maxlen=400)

In [7]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 400)
x_test shape: (25000, 400)


## 1. 搭建網路

In [8]:
model = Sequential()

In [14]:
model.add(Embedding(5000,
                    50,
                    input_length=400))
model.add(Dropout(0.2))

In [15]:
model.add(Conv1D(250,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))

hint:padding = 'valid'    
"valid"代表只進行有效的convolution，即對邊界數據不處理。    
"same"代表保留邊界處的convolution結果，通常會使輸出shape與输入shape相同。

In [16]:
model.add(GlobalMaxPooling1D())

In [18]:
model.add(Dense(250))
model.add(Dropout(0.2))
model.add(Activation('relu'))

In [19]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

hint:輸出層因為要的是非0即1所以Activation fucntion選用sigmoid較合適。

In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## 2. 訓練開始！
為了節省時間請訓練2-5次就好，等你會到家要訓練幾次隨你高興:)

In [23]:
model.fit(x_train, y_train, batch_size=100, epochs=3)

Epoch 1/3
25000/25000 [==============================] - 204s - loss: 0.4463 - acc: 0.7838   
Epoch 2/3
25000/25000 [==============================] - 200s - loss: 0.2526 - acc: 0.8960   
Epoch 3/3
25000/25000 [==============================] - 203s - loss: 0.1847 - acc: 0.9294   


## 3. 使用偉大的神經網路
* 秀出結果
* model.evaluate

In [24]:
from ipywidgets import interact_manual

In [25]:
result = model.predict_classes(x_test)

25000/25000 [==============================] - 32s    


In [ ]:
model_json = model.to_json()
open('handwriting_model_architecture.json', 'w').write(model_json)
model.save_weights('0410practice')

In [26]:
score = model.evaluate(x_test,y_test)
print("Total loss on testing set : ",score[0])
print("Accurancy of testing set : ",score[1])

25000/25000 [==============================] - 32s    
Total loss on testing set :  0.279866161652
Accurancy of testing set :  0.88532


[參考網站](https://github.com/fchollet/keras/blob/master/examples/imdb_cnn.py)